In [1]:
import andes
import pandapower as pp
from jams import dcopf

from andes.interop.pandapower import build_group_table, add_gencost, make_GSF, to_pandapower

import numpy as np
import pandas as pd

In [2]:
print(andes.__version__)
print(pp.__version__)

1.7.0.post170+g48367b96
2.8.0


No line limits:
Only linear cost or only quadratic cost, the results are the same.
With line limits:
not the same with pandapower or matpower.

In [3]:
case_path = '/Users/jinningwang/Documents/work/andes/jwang/jams/cases/'
case39 = 'case39.xlsx'
ssa = andes.load(case_path+case39)

In [4]:
ssd = dcopf()

ssd.from_andes(ssa)
ssd.cost.c1 = range(1,ssd.gen.shape[0]+1)
ssd.cost.c2 = 0
ssd.build()
ssd.mdl.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


Successfully build DCOPF model.


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 71 rows, 10 columns and 492 nonzeros
Model fingerprint: 0x5c2ed624
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [5e+00, 1e+01]
  RHS range        [8e-01, 6e+01]
Presolve removed 52 rows and 0 columns
Presolve time: 0.01s
Presolved: 19 rows, 16 columns, 171 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.2542300e+01   1.375001e+02   0.000000e+00      0s
       8    3.1019679e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.101967924e+02


In [19]:
ssp = to_pandapower(ssa)

# add gen cost
gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])

add_gencost(ssp, gen_cost)

ssp.poly_cost['sort'] = ssp.gen.name.values
ssp.poly_cost.sort_values(by=['sort'], inplace=True)
ssp.poly_cost.drop(['sort'], axis=1, inplace=True)

ssp.poly_cost.cp2_eur_per_mw2 = ssd.cost.c2  # c2
ssp.poly_cost.cp1_eur_per_mw = ssd.cost.c1  # c1

pp.rundcopp(ssp)

gen vm_pu > bus max_vm_pu for gens [5]. Setting bus limit for these gens.


In [28]:
ssdcost = ssd.cost[['idx', 'c1', 'c2']]
ssdcost.idx = ssdcost.idx - 1
ssdcost.rename(columns={'idx': 'element'}, inplace=True)
costdata = pd.merge(ssp.poly_cost, ssdcost[['element', 'c1', 'c2']], on='element')

ssp.poly_cost.cp2_eur_per_mw2 = costdata.c2  # c2
ssp.poly_cost.cp1_eur_per_mw = costdata.c1  # c1

In [29]:
pp.runopp(ssp)
res_pp = ssp.res_gen
res_jm = ssd.get_res()
res_comp = pd.concat([res_pp, res_jm], axis=1)
res_comp.pg = res_comp.pg * ssd.mva

res_comp

gen vm_pu > bus max_vm_pu for gens [5]. Setting bus limit for these gens.


,p_mw,q_mvar,va_degree,vm_pu,gen,pg
0,1039.999997,229.799731,10.212473,1.053922,1,1040.000000
1,311.282959,223.438736,-4.811493,1.032112,3,725.000000
2,0.000030,79.463638,-11.979527,1.001326,4,652.000000
3,507.999996,151.165745,-8.187319,1.016167,5,508.000000
4,686.999995,228.559348,2.256983,1.060000,6,687.000000
5,579.999994,62.405362,5.031877,1.060000,7,580.000000
6,563.999993,52.368137,7.640819,1.041438,8,363.987128
7,864.999989,44.285048,10.423980,1.039312,9,808.262018
8,1099.999986,33.248247,-6.747314,1.039387,10,287.135345
9,645.999973,299.997574,0.000000,1.040991,2,602.845509


In [22]:
ssp.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,gen,0.0,1,0,0.0,0.0,0.0
9,9,gen,0.0,10,0,0.0,0.0,0.0
1,1,gen,0.0,2,0,0.0,0.0,0.0
2,2,gen,0.0,3,0,0.0,0.0,0.0
3,3,gen,0.0,4,0,0.0,0.0,0.0
4,4,gen,0.0,5,0,0.0,0.0,0.0
5,5,gen,0.0,6,0,0.0,0.0,0.0
6,6,gen,0.0,7,0,0.0,0.0,0.0
7,7,gen,0.0,8,0,0.0,0.0,0.0
8,8,gen,0.0,9,0,0.0,0.0,0.0


In [24]:
ssd.cost

,idx,c2,c1,c0,cr
0,1,0,1,0,0
1,3,0,2,0,0
2,4,0,3,0,0
3,5,0,4,0,0
4,6,0,5,0,0
5,7,0,6,0,0
6,8,0,7,0,0
7,9,0,8,0,0
8,10,0,9,0,0
9,2,0,10,0,0
